In [1]:
import pandas as pd

df= pd.read_csv("s3://fr-detector/balanced_train/train_final_1.csv")
 
df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,record_id
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,-0.353703,0,0
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,-0.200026,0,1
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,-0.345144,0,2
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,0.106129,0,3
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,-0.286215,0,4


In [2]:
df["Class"].describe()

count    341176.000000
mean          0.333332
std           0.471405
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: Class, dtype: float64

In [3]:
df["Class"].value_counts()

0    227451
1    113725
Name: Class, dtype: int64

In [6]:
import pandas as pd
from imblearn.over_sampling import SMOTE
import boto3
import io

In [5]:
!pip install imblearn

In [7]:
# Display class distribution before balancing
print("Class distribution before balancing:")
print(df['Class'].value_counts())

# Separate features and target variable
X = df.drop('Class', axis=1)
y = df['Class']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['Class'] = y_resampled

s3 = boto3.client('s3')
bucket="fr-detector"

# Display class distribution after balancing
print("Class distribution after balancing:")
print(df_resampled['Class'].value_counts())

# Save the balanced dataset back to S3
balanced_file_path = 'balanced_train/train_final_1_balanced.csv'
csv_buffer = io.StringIO()
df_resampled.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket, Key=balanced_file_path, Body=csv_buffer.getvalue())

print(f"Balanced dataset saved to {balanced_file_path} in S3 bucket {bucket}.")

Class distribution before balancing:
0    227451
1    113725
Name: Class, dtype: int64
Class distribution after balancing:
0    227451
1    227451
Name: Class, dtype: int64
Balanced dataset saved to balanced_train/train_final_1_balanced.csv in S3 bucket fr-detector.


In [8]:
df1= pd.read_csv("s3://fr-detector/balanced_train/train_final_1_balanced.csv")
df1.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,record_id,Class
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,-0.353703,0,0
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,-0.200026,1,0
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,-0.345144,2,0
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,0.106129,3,0
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,-0.286215,4,0


In [9]:
model_data=df1
print(model_data.head())
model_data=pd.concat([model_data['Class'],model_data.drop(['Class'],axis=1)],axis=1)
print(model_data.head())                    

       Time        V1        V2        V3        V4        V5        V6  \
0  143352.0  1.955041 -0.380783 -0.315013  0.330155 -0.509374 -0.086197   
1  117173.0 -0.400975 -0.626943  1.555339 -2.017772 -0.107769  0.168310   
2  149565.0  0.072509  0.820566 -0.561351 -0.709897  1.080399 -0.359429   
3   93670.0 -0.535045  1.014587  1.750679  2.769390  0.500089  1.002270   
4   82655.0 -4.026938  1.897371 -0.429786 -0.029571 -0.855751 -0.480406   

         V7        V8        V9  ...       V22       V23       V24       V25  \
0 -0.627978  0.035994  1.054560  ...  0.968305  0.053208 -0.278602 -0.044999   
1  0.017959 -0.401619  0.040378  ...  0.421703  0.113442 -1.004095 -1.176695   
2  0.787858  0.117276 -0.131275  ... -0.872959  0.083391  0.148178 -0.431459   
3  0.847902 -0.081323  0.371579  ...  0.443431 -0.072754  0.448192 -0.655203   
4 -0.435632  1.313760  0.536044  ... -0.230369  0.250717  0.066399  0.470787   

        V26       V27       V28    Amount  record_id  Class  
0 -0.2

In [10]:
import boto3
import os
import sagemaker
import pandas as pd
import numpy as np
session = sagemaker.Session()
 
bucket = "fr-detector"
sagemaker_iam_role = sagemaker.get_execution_role()
 
prefix = 'sagemaker/DEMO-xgboost-fraud'
 
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),
                                                  [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
 
 
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')) \
                                .upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')) \
                                .upload_file('validation.csv')
s3_train_data = 's3://{}/{}/train/train.csv'.format(bucket, prefix)
s3_validation_data = 's3://{}/{}/validation/validation.csv'.format(bucket, prefix)
print('Uploaded training data location: {}'.format(s3_train_data))
print('Uploaded training data location: {}'.format(s3_validation_data))
 
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/26/25 06:05:52] INFO     Found credentials from IAM Role:                                   ]8;id=705549;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=103218;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/26/25 06:06:27] INFO     Found credentials from IAM Role:                                   ]8;id=299935;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=280367;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/26/25 06:06:30] INFO     Found credentials from IAM Role:                                   ]8;id=270147;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=505289;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Uploaded training data location: s3://fr-detector/sagemaker/DEMO-xgboost-fraud/train/train.csv
Uploaded training data location: s3://fr-detector/sagemaker/DEMO-xgboost-fraud/validation/validation.csv
Training artifacts will be uploaded to: s3://fr-detector/sagemaker/DEMO-xgboost-fraud/output


In [11]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')
print(container)

[03/26/25 06:06:40] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=539258;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=551892;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=270211;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=789965;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:1


In [12]:
import sagemaker

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [13]:
xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker_iam_role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=session)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

[03/26/25 06:06:46] WARNING  train_instance_count has been renamed in sagemaker>=2.              ]8;id=813989;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=696673;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_instance_type has been renamed in sagemaker>=2.               ]8;id=72863;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=24450;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

In [14]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

[03/26/25 06:06:50] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=511031;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=869163;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-03-26-06-06-50-937       ]8;id=873467;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=167583;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-26 06:06:56 Starting - Starting the training job...
..25-03-26 06:07:11 Starting - Preparing the instances for training.
..25-03-26 06:07:36 Downloading - Downloading input data.
.....03-26 06:08:11 Downloading - Downloading the training image.
2025-03-26 06:08:57 Training - Training image download completed. Training in progress.Arguments: train
[2025-03-26:06:09:13:INFO] Running standalone xgboost training.
[2025-03-26:06:09:13:INFO] File size need to be processed in the node: 221.1mb. Available memory size in the node: 8566.5mb
[2025-03-26:06:09:13:INFO] Determined delimiter of CSV input is ','
[06:09:13] S3DistributionType set as FullyReplicated
[06:09:13] 318431x31 matrix with 9871361 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-03-26:06:09:13:INFO] Determined delimiter of CSV input is ','
[06:09:13] S3DistributionType set as FullyReplicated
[06:09:13] 90980x31 matrix with 2820380 entries loaded from /opt/ml/input/data/validation

In [17]:
xgb.name = 'deployed-xgboost-fraud-prediction'
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge',
                          endpoint_name='deployed-xgboost-fraud-prediction-1')

[03/26/25 06:13:43] INFO     Creating model with name: xgboost-2025-03-26-06-13-43-119              ]8;id=274861;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=710490;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name deployed-xgboost-fraud-prediction-1 ]8;id=997718;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=280742;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5937\5937]8;;\

[03/26/25 06:13:44] INFO     Creating endpoint with name deployed-xgboost-fraud-prediction-1        ]8;id=464445;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=306199;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4759\4759]8;;\

---------!

In [21]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Initialize the predictor
predictor = Predictor(endpoint_name='deployed-xgboost-fraud-prediction-1',
                      serializer=CSVSerializer(),
                      deserializer=JSONDeserializer())

In [22]:
import numpy as np

# Create a sample input (replace with actual feature values)
sample_input = np.array([143352.0, 1.955041, -0.380783, -0.315013, 0.330155, -0.509374, -0.086197, -0.627978, 0.035994,
                         0.099905, 0.172726, 0.338682, -0.119083, 0.048724, 0.292383, -0.287362, 0.014749, 0.027628,
                         -0.003602, 0.022924, 0.238197, 0.968305, 0.053208, -0.278602, -0.044999, -0.216780, 0.045168,
                         -0.047145, -0.353703, 0.0])

# Convert the sample input to CSV format
sample_input_csv = ','.join(map(str, sample_input.flatten()))

In [23]:
# Make a prediction
response = predictor.predict(sample_input_csv)

# Print the prediction result
print("Prediction result:", response)

Prediction result: 6.79199956721277e-06


In [24]:
import boto3
import pandas as pd
from io import StringIO

# Define the s3 client
s3 = boto3.client('s3')

# Load the test data from S3
bucket = "fr-detector"
test_file_path = 'sagemaker/DEMO-xgboost-fraud/validation/validation.csv'
csv_obj = s3.get_object(Bucket=bucket, Key=test_file_path)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

# Create DataFrame
test_data=pd.read_csv("s3://fr-detector/sagemaker/DEMO-xgboost-fraud/validation/validation.csv")
# test_data = pd.read_csv(StringIO(csv_string), header=None)
print(test_data.columns)

Index(['1', '152138.780957399', '-3.943709728863173', '3.655442876244675',
       '-7.486669086917897', '7.473230634292371', '-0.6835801694932635',
       '-1.8939831612320632', '-3.2545157793510877', '0.1352220321826953',
       '-1.558818129300817', '-2.196146215658774', '4.243414845450669',
       '-8.296941023002969', '-1.1078385623297131', '-12.941164483122332',
       '1.0479291709709675', '-4.087546427779548', '-5.541397554601554',
       '-1.175686985762699', '0.9649178547554276', '-0.1786712665954623',
       '0.5300198046342766', '0.6891991985163467', '0.4003281717075747',
       '-0.5432556713754301', '-0.3497788369665653', '0.3431675477985005',
       '-2.533513786094396', '0.3808542921639663', '-0.3918231622046159',
       '270536'],
      dtype='object')


In [21]:
import pandas as pd

# Load the test data from S3 without headers
test_data = pd.read_csv('s3://fr-detector/sagemaker/DEMO-xgboost-fraud/validation/validation.csv', header=None)

In [22]:
# Check for unique values in each column
unique_values = test_data.nunique()

# Print the unique values for each column
print(unique_values)

0         2
1     83293
2     89979
3     89979
4     89979
5     89979
6     89979
7     89979
8     89979
9     89979
10    89979
11    89979
12    89979
13    89979
14    89979
15    89979
16    89979
17    89979
18    89979
19    89979
20    89979
21    89979
22    89979
23    89979
24    89979
25    89979
26    89979
27    89979
28    89979
29    89979
30    50389
31    84699
dtype: int64


In [26]:
column_names = ['Class', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'record_id']

# Assign column names to the test data
test_data.columns = column_names

In [27]:
# Separate features and target variable
X_test = test_data.drop('Class', axis=1)
y_test = test_data['Class']

In [29]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = Predictor(endpoint_name='deployed-xgboost-fraud-prediction-1',
                      serializer=CSVSerializer(),
                      deserializer=JSONDeserializer())

# Make predictions
y_pred = []
for index, row in X_test.iterrows():
    sample_input_csv = ','.join(map(str, row.values))
    response = predictor.predict(sample_input_csv)
    y_pred.append(float(response))

# Convert predictions to binary (0 or 1) based on a threshold (e.g., 0.5)
y_pred_binary = [1 if prob > 0.5 else 0 for prob in y_pred]

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_test, y_pred_binary)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, y_pred_binary)
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_test, y_pred_binary)
print(f"F1 Score: {f1}")

Accuracy: 0.9996922366700007
Precision: 0.9998247497206949
Recall: 0.9995619894439456
F1 Score: 0.9996933523162852


In [ ]:
# batch infrencing 

In [38]:
from sagemaker.transformer import Transformer

transformer = Transformer(
    model_name='xgboost-2025-03-26-06-13-43-119',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path='s3://fr-detector/sagemaker/DEMO-xgboost-fraud/batch-output',
    strategy='SingleRecord'
)

transformer.transform(
    data='s3://fr-detector/sagemaker/DEMO-xgboost-fraud/batch-input',
    content_type='text/csv',
    split_type='Line'
)

transformer.wait()

[03/26/25 06:36:39] INFO     Creating transform job with name: xgboost-2025-03-26-06-36-39-807      ]8;id=48639;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=750347;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#3951\3951]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   strategy='SingleRecord'                                                                 │
│    9 )                                                                                           │
│   10                                                                                             │
│ ❱ 11 transformer.transform(                                                                      │
│   12 │   data='s3://fr-detector/sagemaker/DEMO-xgboost-fraud/batch-input',                       │
│   13 │   content_type='text/csv',                                                                │
│   14 │   split_type='Line'                                                                       │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/transformer.py:302  │
│ in transform                                                                                     │
│                                                                                                  │
│   299 │   │   │   sagemaker_session=self.sagemaker_session,                                      │
│   300 │   │   )                                                                                  │
│   301 │   │                                                                                      │
│ ❱ 302 │   │   self.latest_transform_job = _TransformJob.start_new(                               │
│   303 │   │   │   self,                                                                          │
│   304 │   │   │   data,                                                                          │
│   305 │   │   │   data_type,                                                                     │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/transformer.py:636  │
│ in start_new                                                                                     │
│                                                                                                  │
│   633 │   │   │   batch_data_capture_config,                                                     │
│   634 │   │   )                                                                                  │
│   635 │   │                                                                                      │
│ ❱ 636 │   │   transformer.sagemaker_session.transform(**tra

In [ ]:
# Data and Model Monitoring

In [37]:
from sagemaker.model_monitor import DefaultModelMonitor

monitor = DefaultModelMonitor(
    role=sagemaker_iam_role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=100,
    max_runtime_in_seconds=3600
)

monitor.create_monitoring_schedule(
    endpoint_input='deployed-xgboost-fraud-prediction-1',
    output_s3_uri='s3://fr-detector/sagemaker/DEMO-xgboost-fraud/monitoring-output',
    schedule_cron_expression='cron(0 * ? * * *)'
)

[03/26/25 06:36:15] INFO     Ignoring unnecessary instance type: None.                            ]8;id=842455;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=931747;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   max_runtime_in_seconds=3600                                                             │
│    9 )                                                                                           │
│   10                                                                                             │
│ ❱ 11 monitor.create_monitoring_schedule(                                                         │
│   12 │   endpoint_input='deployed-xgboost-fraud-prediction-1',                                   │
│   13 │   output_s3_uri='s3://fr-detector/sagemaker/DEMO-xgboost-fraud/monitoring-output',        │
│   14 │   schedule_cron_expression='cron(0 * ? * * *)'                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model │
│ _monitoring.py:2046 in create_monitoring_schedule                                                │
│                                                                                                  │
│   2043 │   │   │   network_config=self.network_config,                                           │
│   2044 │   │   │   batch_transform_input=batch_transform_input,                                  │
│   2045 │   │   )                                                                                 │
│ ❱ 2046 │   │   self.sagemaker_session.sagemaker_client.create_data_quality_job_definition(**req  │
│   2047 │   │                                                                                     │
│   2048 │   │   # create schedule                                                                 │
│   2049 │   │   try:                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:570 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:124 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)         

In [ ]:
# Email Alerts for Drift Detection

In [ ]:
import boto3

sns = boto3.client('sns')
response = sns.create_topic(Name='ModelDriftAlerts')
topic_arn = response['TopicArn']

In [ ]:
#Subscribe to the Topic:
response = sns.subscribe(
    TopicArn=topic_arn,
    Protocol='email',
    Endpoint='your-email@example.com'
)

In [ ]:
#Create a Lambda Function:
import json
import boto3

def lambda_handler(event, context):
    sns = boto3.client('sns')
    message = 'Data drift detected in the model.'
    sns.publish(TopicArn='your-topic-arn', Message=message)

In [ ]:
#  CI/CD Pipeline
# Use AWS CodePipeline to create a CI/CD pipeline:

# Create a CodePipeline:
import boto3

codepipeline = boto3.client('codepipeline')
response = codepipeline.create_pipeline(
    pipeline={
        'name': 'FraudDetectionPipeline',
        'roleArn': 'your-codepipeline-role-arn',
        'stages': [
            {
                'name': 'Source',
                'actions': [
                    {
                        'name': 'Source',
                        'actionTypeId': {
                            'category': 'Source',
                            'owner': 'AWS',
                            'provider': 'CodeCommit',
                            'version': '1'
                        },
                        'outputArtifacts': [{'name': 'SourceArtifact'}],
                        'configuration': {
                                           ]
            },
            {
                'name': 'Build',
                'actions': [
                    {
                        'name': 'Build',
                        'actionTypeId': {
                            'category': 'Build',
                            'owner': 'AWS',
                            'provider': 'CodeBuild',
                            'version': '1'
                        },
                        'inputArtifacts': [{'name': 'SourceArtifact'}],
                        'outputArtifacts': [{'name': 'BuildArtifact'}],
                        'configuration': {
                            'ProjectName': 'your-codebuild-project-name'
                        }
                    }
                ]
            },
            {
                'name': 'Deploy',
                'actions': [
                    {
                        'namecategory': 'Deploy',
                            'owner': 'AWS',
                            'provider': 'CloudFormation',
                            'version': '1'
                        },
                        'inputArtifacts': [{'name': 'BuildArtifact'}],
                        'configuration': {
                            'ActionMode': 'CHANGE_SET_REPLACE',
                            'StackName': 'your-stack-name',
                            'ChangeSetName': 'your-changeset-name'
                        }
                    }
                ]
            }
        ]
    }
)